In [21]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [164]:
indices = ['id', 'tag']
left = df2.set_index(indices)
right = df1.set_index(indices)
combined = left.combine_first(right).reset_index()
combined

,id,tag,Membership
0,1,700,0.5
1,1,800,0.9
2,2,700,0.8
3,2,800,0.9
4,3,500,0.7
5,3,600,0.8


In [165]:
indices = ['id', 'tag']
df2.set_index(indices).combine_first(df1.set_index(indices)).reset_index()

,id,tag,Membership
0,1,700,0.5
1,1,800,0.9
2,2,700,0.8
3,2,800,0.9
4,3,500,0.7
5,3,600,0.8


In [146]:
df = df1.merge(df2, on=['id', 'tag'], how='left')
df = df[df['Membership_y'].isna()].drop(
    columns=['Membership_y']).rename(columns={'Membership_x':'Membership'})
df2 = df2.append(df).reset_index(drop=True)
df2

,id,tag,Membership
0,1,700,0.5
1,2,800,0.9
2,3,600,0.8
3,3,500,0.7
4,1,800,0.9
5,2,700,0.8


In [148]:
d="""date
2021-05-21 00:02:11.349001429
2021-05-21 00:02:38.195857153
2021-05-21 00:03:25.527530228
2021-05-21 00:03:26.653410069
2021-05-21 00:03:26.798157366"""
df1=g(d)
df1

,date
0,2021-05-21 00:02:11.349001429
1,2021-05-21 00:02:38.195857153
2,2021-05-21 00:03:25.527530228
3,2021-05-21 00:03:26.653410069
4,2021-05-21 00:03:26.798157366


In [149]:
d="""date
2021-05-21 00:02:11.348997322
2021-05-21 00:02:38.195852267
2021-05-21 00:03:25.527526087
2021-05-21 00:03:26.653406759
2021-05-21 00:03:26.798154350"""
df2=g(d)
df2

,date
0,2021-05-21 00:02:11.348997322
1,2021-05-21 00:02:38.195852267
2,2021-05-21 00:03:25.527526087
3,2021-05-21 00:03:26.653406759
4,2021-05-21 00:03:26.798154350


In [150]:
df1.date = pd.to_datetime(df1.date)
df2.date = pd.to_datetime(df2.date)
df1

,date
0,2021-05-21 00:02:11.349001429
1,2021-05-21 00:02:38.195857153
2,2021-05-21 00:03:25.527530228
3,2021-05-21 00:03:26.653410069
4,2021-05-21 00:03:26.798157366


In [156]:
s1 = df1.squeeze()
s2 = df2.squeeze()

In [157]:
s1 - s2

0   0 days 00:00:00.000004107
1   0 days 00:00:00.000004886
2   0 days 00:00:00.000004141
3   0 days 00:00:00.000003310
4   0 days 00:00:00.000003016
Name: date, dtype: timedelta64[ns]

In [158]:
d="""check_NUMBER,2021-12-01,2022-01-01,2022-02-01
1000M,24,17,22
1000M,24,83,55"""
df=g(d)
df

,check_NUMBER,2021-12-01,2022-01-01,2022-02-01
0,1000M,24,17,22
1,1000M,24,83,55


In [162]:
df.melt(id_vars='check_NUMBER', var_name='dates', value_name='values')

,check_NUMBER,dates,values
0,1000M,2021-12-01,24
1,1000M,2021-12-01,24
2,1000M,2022-01-01,17
3,1000M,2022-01-01,83
4,1000M,2022-02-01,22
5,1000M,2022-02-01,55


In [166]:
d="""date,value
2021-04-29,1
2021-05-03,2
2021-05-06,1
2021-05-15,1
2021-05-17,2
2021-05-18,1
2021-05-21,2
2021-05-22,5
2021-05-24,4"""
df=g(d)
df

,date,value
0,2021-04-29,1
1,2021-05-03,2
2,2021-05-06,1
3,2021-05-15,1
4,2021-05-17,2
5,2021-05-18,1
6,2021-05-21,2
7,2021-05-22,5
8,2021-05-24,4


In [167]:
df.date = pd.to_datetime(df.date)
df

,date,value
0,2021-04-29,1
1,2021-05-03,2
2,2021-05-06,1
3,2021-05-15,1
4,2021-05-17,2
5,2021-05-18,1
6,2021-05-21,2
7,2021-05-22,5
8,2021-05-24,4


In [168]:
import pandas as pd

cars2 = {'ID': [22,100,47,35,60],
    'Date': ['2020-04-11','2021-04-12','2020-05-13','2020-05-14', '2020-06-15'],
        'diagnosis': ['bacteria sepsis','bacteria sepsis','Sepsis','Risk sepsis','Neonatal sepsis'],
        'outcome': ['alive','alive','dead','alive','dead']
        
        }

df2 = pd.DataFrame(cars2, columns = ['ID','Date', 'diagnosis', 'outcome'])

print (df2)

    ID        Date        diagnosis outcome
0   22  2020-04-11  bacteria sepsis   alive
1  100  2021-04-12  bacteria sepsis   alive
2   47  2020-05-13           Sepsis    dead
3   35  2020-05-14      Risk sepsis   alive
4   60  2020-06-15  Neonatal sepsis    dead


In [184]:
df = pd.DataFrame({
    'date':pd.date_range(start='2020-05-24',
                         periods=6, freq='1Y')
})
df

,date
0,2020-12-31
1,2021-12-31
2,2022-12-31
3,2023-12-31
4,2024-12-31
5,2025-12-31


In [185]:
d="""datetime,id,name
2004-04-01 09:45:00,5,bypedal
2004-04-01 09:50:00,5,bypedal
2004-04-01 09:55:00,5,bypedal
2004-04-01 10:00:00,5,bypedal
2004-04-01 10:05:00,5,bypedal"""
df=g(d)
df

,datetime,id,name
0,2004-04-01 09:45:00,5,bypedal
1,2004-04-01 09:50:00,5,bypedal
2,2004-04-01 09:55:00,5,bypedal
3,2004-04-01 10:00:00,5,bypedal
4,2004-04-01 10:05:00,5,bypedal


In [186]:
df.datetime = pd.to_datetime(df.datetime)
df = df.set_index('datetime')
df

,id,name
datetime,,
2004-04-01 09:45:00,5,bypedal
2004-04-01 09:50:00,5,bypedal
2004-04-01 09:55:00,5,bypedal
2004-04-01 10:00:00,5,bypedal
2004-04-01 10:05:00,5,bypedal


In [198]:
np.where((df.between_time('14:00', '10:00')), 1, 0)

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [5]:
d="""Date,Portfoliovalue
2021-05-01,50000.0
2021-05-02,52304.0"""
df=g(d)
df

,Date,Portfoliovalue
0,2021-05-01,50000.0
1,2021-05-02,52304.0


In [6]:
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date').asfreq('D').reset_index()

In [7]:
df

,Date,Portfoliovalue
0,2021-05-01,50000.0
1,2021-05-02,52304.0


In [8]:
d="""id,text
1,one two three
2,one two
3,four
4,six seven
5,84
6,4x2"""
df=g(d)
df

,id,text
0,1,one two three
1,2,one two
2,3,four
3,4,six seven
4,5,84
5,6,4x2


In [10]:
df.text.str.split(' ').str[0]

0     one
1     one
2    four
3     six
4      84
5     4x2
Name: text, dtype: object

In [8]:
d = """Year,Week_Number,open,high,low,close,volume,market_cap
2020,31,11106.793367,12041.230145,10914.007709,11059.660924,86939673211,836299315108
2020,32,11059.658520,11903.881608,11011.841384,11653.660942,125051146775,1483987715241
2020,33,11665.874956,12047.515879,11199.052457,11906.236593,141819289223,1513036354035
2020,34,11915.898402,12382.422676,11435.685834,11671.520767,136888268138,1533135548697
2020,35,11668.211439,11806.669046,11183.114210,11704.963980,122232543594,1490089199926
2020,36,11713.540300,12044.196936,9951.201578,10277.329333,161912442921,1434502733759"""
df=g(d)
df

,Year,Week_Number,open,high,low,close,volume,market_cap
0,2020,31,11106.793367,12041.230145,10914.007709,11059.660924,86939673211,836299315108
1,2020,32,11059.658520,11903.881608,11011.841384,11653.660942,125051146775,1483987715241
2,2020,33,11665.874956,12047.515879,11199.052457,11906.236593,141819289223,1513036354035
3,2020,34,11915.898402,12382.422676,11435.685834,11671.520767,136888268138,1533135548697
4,2020,35,11668.211439,11806.669046,11183.114210,11704.963980,122232543594,1490089199926
5,2020,36,11713.540300,12044.196936,9951.201578,10277.329333,161912442921,1434502733759


In [6]:
df['week_date'] = pd.to_datetime(
    df['Year'].astype(str) + df['Week_Number'].astype(str) + "1",
    format='%G%V%w'
)

In [7]:
df

,Year,Week_Number,open,high,low,close,volume,market_cap,week_date
0,2020,31,11106.793367,12041.230145,10914.007709,11059.660924,86939673211,836299315108,2020-07-27
1,2020,32,11059.658520,11903.881608,11011.841384,11653.660942,125051146775,1483987715241,2020-08-03
2,2020,33,11665.874956,12047.515879,11199.052457,11906.236593,141819289223,1513036354035,2020-08-10
3,2020,34,11915.898402,12382.422676,11435.685834,11671.520767,136888268138,1533135548697,2020-08-17
4,2020,35,11668.211439,11806.669046,11183.114210,11704.963980,122232543594,1490089199926,2020-08-24
5,2020,36,11713.540300,12044.196936,9951.201578,10277.329333,161912442921,1434502733759,2020-08-31


In [12]:
d = """pkey_id,fair_price,date,status,% change,Difference,Comments
465620,45080.23,2021-05-17 12:28:30,idle,1,45080.23,
465625,45060.17,2021-05-17 12:28:35,idle,-0.00045,-20.06,// Generally looks downward
465629,45052.74,2021-05-17 12:28:40,idle,-0.00016,-7.43,// Things are not moving rapidly up or down. It’s just a “gradual” trend without too many reversals
465633,45043.89,2021-05-17 12:28:45,idle,-0.0002,-8.85,// In the previous data you’d see a few big swings. That would increase my risk
465636,45040.93,2021-05-17 12:28:50,idle,-0.00007,-2.96,// Big swings against the trend and position would red flag me
465640,45049.95,2021-05-17 12:28:56,idle,0.0002,9.02,
465643,45045.38,2021-05-17 12:29:00,idle,-0.0001,-4.57,
465646,45039.87,2021-05-17 12:29:05,idle,-0.00012,-5.51,
465650,45045.55,2021-05-17 12:29:10,idle,0.00013,5.68,
465652,45042.53,2021-05-17 12:29:15,idle,-0.00007,-3.02,
465653,45039.34,2021-05-17 12:29:20,idle,-0.00007,-3.19,
465654,45039.33,2021-05-17 12:29:25,idle,0,-0.009999999995,
465657,45045.77,2021-05-17 12:29:30,idle,0.00014,6.44,
465672,45062.57,2021-05-17 12:29:36,idle,0.00037,16.8,
465676,45033.99,2021-05-17 12:29:40,idle,-0.00063,-28.58,
465678,45032.92,2021-05-17 12:29:45,idle,-0.00002,-1.07,
465679,45037.42,2021-05-17 12:29:50,idle,0.0001,4.5,
465680,45020.85,2021-05-17 12:29:55,idle,-0.00037,-16.57,
465681,45010.4,2021-05-17 12:30:00,idle,-0.00023,-10.45,
465693,45014.59,2021-05-17 12:30:06,idle,0.00009,4.19,
465695,45021.08,2021-05-17 12:30:10,idle,0.00014,6.49,
465696,45028.86,2021-05-17 12:30:15,idle,0.00017,7.78,
465705,45045.26,2021-05-17 12:30:20,idle,0.00036,16.4,
465707,45053.53,2021-05-17 12:30:25,idle,0.00018,8.27,
465710,45060.88,2021-05-17 12:30:30,idle,0.00016,7.35,
465713,45068.97,2021-05-17 12:30:35,idle,0.00018,8.09,
465714,45064.79,2021-05-17 12:30:40,idle,-0.00009,-4.18,
465719,45052.92,2021-05-17 12:30:45,idle,-0.00026,-11.87,
465724,45041.32,2021-05-17 12:30:50,idle,-0.00026,-11.6,
465728,45029.61,2021-05-17 12:30:55,idle,-0.00026,-11.71,
465731,45028.72,2021-05-17 12:31:01,idle,-0.00002,-0.89,
465740,45045.86,2021-05-17 12:31:05,idle,0.00038,17.14,
465742,45062.62,2021-05-17 12:31:10,idle,0.00037,16.76,
465744,45056.03,2021-05-17 12:31:15,idle,-0.00015,-6.59,
465753,45062.02,2021-05-17 12:31:20,idle,0.00013,5.99,
465755,45066.76,2021-05-17 12:31:25,idle,0.00011,4.74,
465761,45081.07,2021-05-17 12:31:30,idle,0.00032,14.31,
465765,45072.8,2021-05-17 12:31:35,idle,-0.00018,-8.27,
465771,45054.23,2021-05-17 12:31:40,idle,-0.00041,-18.57,
465775,45059.88,2021-05-17 12:31:45,idle,0.00013,5.65,
465778,45070.06,2021-05-17 12:31:50,idle,0.00023,10.18,
465781,45073.69,2021-05-17 12:31:55,idle,0.00008,3.63,
465783,45070.33,2021-05-17 12:32:00,idle,-0.00007,-3.36,
465785,45077.86,2021-05-17 12:32:05,idle,0.00017,7.53,
465787,45071.84,2021-05-17 12:32:10,idle,-0.00013,-6.02,
465790,45078.72,2021-05-17 12:32:15,idle,0.00015,6.88,
465798,45093.59,2021-05-17 12:32:20,idle,0.00033,14.87,
465805,45104.14,2021-05-17 12:32:25,idle,0.00023,10.55,
465809,45103.99,2021-05-17 12:32:30,idle,0,-0.15,
465812,45113.27,2021-05-17 12:32:35,idle,0.00021,9.28,
465823,45127.42,2021-05-17 12:32:40,idle,0.00031,14.15,
465830,45130.36,2021-05-17 12:32:45,idle,0.00007,2.94,
465831,45150.51,2021-05-17 12:32:50,idle,0.00045,20.15,
465833,45153.03,2021-05-17 12:32:55,idle,0.00006,2.52,
465834,45149.44,2021-05-17 12:33:00,idle,-0.00008,-3.59,
465835,45152.01,2021-05-17 12:33:05,idle,0.00006,2.57,
465841,45164.2,2021-05-17 12:33:10,idle,0.00027,12.19,
465843,45155.73,2021-05-17 12:33:15,idle,-0.00019,-8.47,
465851,45146.19,2021-05-17 12:33:20,idle,-0.00021,-9.54,
465858,45126.15,2021-05-17 12:33:25,idle,-0.00044,-20.04,
465860,45127.21,2021-05-17 12:33:30,idle,0.00002,1.06,
465868,45095.98,2021-05-17 12:33:36,idle,-0.00069,-31.23,
465870,45087.57,2021-05-17 12:33:40,idle,-0.00019,-8.41,
465871,45083.1,2021-05-17 12:33:45,idle,-0.0001,-4.47,
465872,45077.14,2021-05-17 12:33:50,idle,-0.00013,-5.96,
465875,45081.17,2021-05-17 12:33:55,idle,0.00009,4.03,
465885,45072.33,2021-05-17 12:34:00,idle,-0.0002,-8.84,
465897,45057.09,2021-05-17 12:34:05,idle,-0.00034,-15.24,
465900,45047.82,2021-05-17 12:34:10,idle,-0.00021,-9.27,
465911,45059.55,2021-05-17 12:34:16,idle,0.00026,11.73,
465923,45074.56,2021-05-17 12:34:20,idle,0.00033,15.01,
465934,45108.81,2021-05-17 12:34:25,idle,0.00076,34.25,
465944,45120.73,2021-05-17 12:34:30,idle,0.00026,11.92,
465950,45123.3,2021-05-17 12:34:35,idle,0.00006,2.57,
465953,45100.51,2021-05-17 12:34:40,idle,-0.00051,-22.79,
465958,45090.19,2021-05-17 12:34:45,idle,-0.00023,-10.32,
465965,45088.23,2021-05-17 12:34:50,idle,-0.00004,-1.96,
465975,45112.13,2021-05-17 12:34:55,idle,0.00053,23.9,
465985,45129.68,2021-05-17 12:35:00,idle,0.00039,17.55,
465988,45126.63,2021-05-17 12:35:05,idle,-0.00007,-3.05,
465993,45136.73,2021-05-17 12:35:10,idle,0.00022,10.1,
465997,45139.98,2021-05-17 12:35:16,idle,0.00007,3.25,"""
df=g(d)
df.head()

,pkey_id,fair_price,date,status,% change,Difference,Comments
0,465620,45080.23,2021-05-17 12:28:30,idle,1.00000,45080.23,NaN
1,465625,45060.17,2021-05-17 12:28:35,idle,-0.00045,-20.06,// Generally looks downward
2,465629,45052.74,2021-05-17 12:28:40,idle,-0.00016,-7.43,// Things are not moving rapidly up or down. I...
3,465633,45043.89,2021-05-17 12:28:45,idle,-0.00020,-8.85,// In the previous data you’d see a few big sw...
4,465636,45040.93,2021-05-17 12:28:50,idle,-0.00007,-2.96,// Big swings against the trend and position w...


In [13]:
d="""DTE,Strike,Bid,Ask
1,100,10,11
1,200,16,17
1,300,17,18
1,400,11,12
1,500,12,13
1,600,13,14
2,100,10,30
2,200,15,20
2,300,16,21"""
df=g(d)
df

,DTE,Strike,Bid,Ask
0,1,100,10,11
1,1,200,16,17
2,1,300,17,18
3,1,400,11,12
4,1,500,12,13
5,1,600,13,14
6,2,100,10,30
7,2,200,15,20
8,2,300,16,21


In [45]:
d="""date,Val1,val2,val3,val4,val5,val6,val7
01-02-2021,0,1,3,5,0,0,0
01-02-2021,1,0,0,5,1,4,5
01-02-2021,0,0,2,0,5,0,0
01-02-2021,0,1,3,5,0,0,0
01-02-2021,1,0,0,5,1,4,5
01-02-2021,0,0,2,0,5,0,0"""
df=g(d)
df.index = ['Juan', 'Juan', 'Juan','Pedro','John','John']
df

,date,Val1,val2,val3,val4,val5,val6,val7
Juan,01-02-2021,0,1,3,5,0,0,0
Juan,01-02-2021,1,0,0,5,1,4,5
Juan,01-02-2021,0,0,2,0,5,0,0
Pedro,01-02-2021,0,1,3,5,0,0,0
John,01-02-2021,1,0,0,5,1,4,5
John,01-02-2021,0,0,2,0,5,0,0


In [46]:
df.groupby(level=0, sort=False).sum()

,Val1,val2,val3,val4,val5,val6,val7
Juan,1,1,5,10,6,4,5
Pedro,0,1,3,5,0,0,0
John,1,0,2,5,6,4,5


In [47]:
df = df.reset_index().rename(columns={'index':'name'})
df.groupby(['name', 'date'], sort=False).sum().reset_index(level=1).rename_axis(None)

,date,Val1,val2,val3,val4,val5,val6,val7
Juan,01-02-2021,1,1,5,10,6,4,5
Pedro,01-02-2021,0,1,3,5,0,0,0
John,01-02-2021,1,0,2,5,6,4,5


In [97]:
d="""Date,price,5_Days_Up
20-May-21,1,
21-May-21,2,
22-May-21,4,
23-May-21,5,
24-May-21,6,5
25-May-21,7,6
26-May-21,8,7
27-May-21,9,8
28-May-21,10,9
29-May-21,11,10
30-May-21,12,11
31-May-21,13,12
1-Jun-21,14,13
2-Jun-21,15,14"""
df=g(d)
df.head(7)

,Date,price,5_Days_Up
0,20-May-21,1,NaN
1,21-May-21,2,NaN
2,22-May-21,4,NaN
3,23-May-21,5,NaN
4,24-May-21,6,5.0
5,25-May-21,7,6.0
6,26-May-21,8,7.0


In [98]:
#df.Date = pd.to_datetime(df.Date)
df['5_Days_Up'] = df.rolling(4).price.max().shift()
df

,Date,price,5_Days_Up
0,20-May-21,1,NaN
1,21-May-21,2,NaN
2,22-May-21,4,NaN
3,23-May-21,5,NaN
4,24-May-21,6,5.0
5,25-May-21,7,6.0
6,26-May-21,8,7.0
7,27-May-21,9,8.0
8,28-May-21,10,9.0
9,29-May-21,11,10.0


In [108]:
df = pd.read_csv('te.csv')
df.head()

,candle_date,open,high,low,close,volume,market_cap
0,2020-07-30,11106.793367,11162.524034,10914.007709,11110.450186,18100631315,204878131809
1,2020-07-31,11105.227653,11419.049872,10984.294767,11332.065740,19654692416,204858623147
2,2020-08-01,11325.848010,11836.148677,11244.907247,11783.011541,23928666996,208939244082
3,2020-08-02,11796.061235,12041.230145,11020.410318,11059.660924,25255682484,217623316070
4,2020-08-03,11059.658520,11458.544207,11011.841384,11255.903225,17810127207,204047658291


In [116]:
import datetime
def convert_dailydata_to_weeklydata(daily_data):
    # Print function name
    #print_func_name("convert_dailydata_to_weeklydata")

    # Loop over the rows until a row with Monday as date is present
    row_counter_start = 0
    while True:
        if datetime.date.weekday(daily_data['candle_date'][row_counter_start]) == 0:
            break
        row_counter_start += 1

    # # Loop over the rows until a row with Sunday as date is present
    # row_counter_end = len(daily_data.index) - 1
    # while True:
    #     if datetime.weekday(daily_data['candle_date'][row_counter_end]) == 6:
    #         break
    #     row_counter_end -= 1
    # print(daily_data)
    # print(row_counter_end)

    # Copy all rows after the first Monday row of data is reached
    daily_data_temp = daily_data[row_counter_start:]

    # Getting week number
    daily_data_temp['Week_Number'] = pd.to_datetime(daily_data_temp['candle_date']).dt.week

    # Getting year. Weeknum is common across years to we need to create unique index by using year and weeknum
    daily_data_temp['Year'] = pd.to_datetime(daily_data_temp['candle_date']).dt.year

    # Grouping based on required values
    df = daily_data_temp.groupby(['Year', 'Week_Number']).agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum', 'market_cap': 'sum'})

    # Reset index
    df = df.reset_index()

    # Create week date (start of week)
    # The + "1" isfor the day of the week.Week numbers 0-6 with 0 being Sunday and 6 being Saturday.
    df['week_date'] = pd.to_datetime(df['Year'].astype(str) + df['Week_Number'].astype(str) + "1", format='%G%V%w')

    # Set indexes
    df = df.set_index(['Year', 'Week_Number'])

    # Re-order columns into a new dataframe
    weekly_data = df[["week_date", "open", "high", "low", "close", "volume", "market_cap"]]
    weekly_data = weekly_data.rename({'week_date': 'candle_date'}, axis=1)

    # Drop index columns
    weekly_data.reset_index(drop=True, inplace=True)

    return weekly_data

In [113]:
convert_dailydata_to_weeklydata(df)

AttributeError: module 'datetime' has no attribute 'weekday'